In [1]:
import os
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
from tqdm import tqdm
from scripts import tools
from scripts import dataset
from scripts.config import *
from models import *
from tensorflow.contrib.data import Iterator
from tensorflow.contrib.data import Dataset
from tensorflow.python.framework import dtypes
from tensorflow.python.framework.ops import convert_to_tensor

%matplotlib inline

Joint_Classes = np.array(["Right ankle", "Right knee", "Right hip", "Left hip", "Left knee", "Left ankle", "Right wrist",
                  "Right elbow", "Right shoulder", "Left shoulder", "Left elbow", "Left wrist", "Neck", "Head top"])

MET_Classes = np.array(["walking about", "writing", "reading.seated", "typing", "lifting.packing-lifting", "lifting.packing-packing", 
                        "filling.seated", "filling.stand", "cooking", "house cleaning", "machine work.light", "machine work.sawing", 
                        "reclining", "seated.quiet", "sleeping", "standing.relaxed"])

tools.etc.set_GPU("3")

In [2]:
batch_size=128
learning_rate = 0.01
num_joints = len(Joint_Classes)
num_met = len(MET_Classes)

display_step = 100

PROJECT_ROOT = %pwd

train_csv = PROJECT_ROOT + "/dataset/train.csv"
test_csv = PROJECT_ROOT + "/dataset/test.csv"

if not os.path.isdir(checkpoint_path):
    os.mkdir(checkpoint_path)

In [3]:
def _parse_function_train(img, label):
    return img, tf.one_hot(label, num_met)

In [4]:
train_data = dataset.met(csv_file="./dataset/train.csv", Shuffle=True)
test_data = dataset.met(csv_file="./dataset/test.csv", Shuffle=True)

[Processing Images & Coordinates]: 100%|██████████| 96/96 [00:02<00:00, 44.74it/s] 


In [5]:
tr_img_set = convert_to_tensor(train_data.img_set, dtype=dtypes.float64)
tr_labels = convert_to_tensor(train_data.labels[:,0], dtype= dtypes.int32)

tr_data = Dataset.from_tensor_slices((tr_img_set, tr_labels))

tr_data = tr_data.map(_parse_function_train, num_threads=8, output_buffer_size = 100* batch_size)
tr_data = tr_data.batch(batch_size)

tr_iterator = Iterator.from_structure(tr_data.output_types, tr_data.output_shapes)
training_init_op = tr_iterator.make_initializer(tr_data)

Instructions for updating:
Use `tf.data.Dataset.from_tensor_slices()`.
Instructions for updating:
Replace `num_threads=T` with `num_parallel_calls=T`. Replace `output_buffer_size=N` with `ds.prefetch(N)` on the returned dataset.
Instructions for updating:
Replace `num_threads=T` with `num_parallel_calls=T`. Replace `output_buffer_size=N` with `ds.prefetch(N)` on the returned dataset.


In [6]:
val_img_set = convert_to_tensor(test_data.img_set, dtype=dtypes.float64)
val_labels = convert_to_tensor(test_data.labels, dtype= dtypes.int32)

val_data = Dataset.from_tensor_slices((val_img_set, val_labels))

val_data = val_data.map(_parse_function_train, num_threads=8,
                output_buffer_size=100*batch_size)
val_data = val_data.batch(batch_size)

val_iterator = Iterator.from_structure(val_data.output_types, val_data.output_shapes)
validation_init_op = val_iterator.make_initializer(val_data)

In [7]:
from models import alexnet

In [8]:
model = alexnet.alexnet(batch_size, (227,227,3), (num_met,))

In [9]:
score = model.fc8

In [10]:
train_layers = ['fc8', 'fc7', 'fc6']
var_list = [v for v in tf.trainable_variables() if v.name.split('/')[0] in train_layers]

In [11]:
with tf.name_scope("cross_ent"):
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=score,labels=model.y_gt))

Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See tf.nn.softmax_cross_entropy_with_logits_v2.



In [13]:
with tf.name_scope("train"):
    optimizer = tf.train.AdadeltaOptimizer(learning_rate=1,rho=0.95,epsilon=1e-09).minimize(loss)
    #train_op = optimizer.apply_gradients(grads_and_vars=gradients)

In [14]:
for gradient, var in gradients:
    tf.summary.histogram(var.name + '/gradient', gradient)

INFO:tensorflow:Summary name fc6/weights:0/gradient is illegal; using fc6/weights_0/gradient instead.
INFO:tensorflow:Summary name fc6/biases:0/gradient is illegal; using fc6/biases_0/gradient instead.
INFO:tensorflow:Summary name fc7/weights:0/gradient is illegal; using fc7/weights_0/gradient instead.
INFO:tensorflow:Summary name fc7/biases:0/gradient is illegal; using fc7/biases_0/gradient instead.
INFO:tensorflow:Summary name fc8/weights:0/gradient is illegal; using fc8/weights_0/gradient instead.
INFO:tensorflow:Summary name fc8/biases:0/gradient is illegal; using fc8/biases_0/gradient instead.


In [15]:
for var in var_list:
    tf.summary.histogram(var.name, var)

INFO:tensorflow:Summary name fc6/weights:0 is illegal; using fc6/weights_0 instead.
INFO:tensorflow:Summary name fc6/biases:0 is illegal; using fc6/biases_0 instead.
INFO:tensorflow:Summary name fc7/weights:0 is illegal; using fc7/weights_0 instead.
INFO:tensorflow:Summary name fc7/biases:0 is illegal; using fc7/biases_0 instead.
INFO:tensorflow:Summary name fc8/weights:0 is illegal; using fc8/weights_0 instead.
INFO:tensorflow:Summary name fc8/biases:0 is illegal; using fc8/biases_0 instead.


In [16]:
tf.summary.scalar('cross_entropy', loss)

<tf.Tensor 'cross_entropy:0' shape=() dtype=string>

In [17]:
with tf.name_scope("accuracy"):
    correct_pred = tf.equal(tf.argmax(score, 1), tf.argmax(model.y_gt, 1))
    accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

In [18]:
tf.summary.scalar('accuracy', accuracy)

<tf.Tensor 'accuracy_1:0' shape=() dtype=string>

In [19]:
merged_summary = tf.summary.merge_all()

In [20]:
writer = tf.summary.FileWriter(filewriter_path)

In [21]:
saver = tf.train.Saver()


In [22]:
train_batches_per_epoch = int(np.floor(len(train_data.img_path)/batch_size))
val_batches_per_epoch = int(np.floor(len(test_data.img_path) / batch_size))

In [23]:
keep_prob = tf.placeholder(tf.float32)

In [ ]:
num_epochs = 1000
next_batch = tr_iterator.get_next()

with tf.device('/gpu:0'):
    with tqdm(total = num_epochs) as pbar:
        with tf.Session() as sess:
            # Initialize all variables
            sess.run(tf.global_variables_initializer())

            # Add the model graph to TensorBoard
            writer.add_graph(sess.graph)

            # Loop over number of epochs
            for epoch in range(num_epochs):

                # Initialize iterator with the training dataset
                sess.run(training_init_op)

                train_acc = 0.
                train_loss = 0.
                train_count = 0
                for step in range(train_batches_per_epoch):
                    # get next batch of data
                    img_batch, label_batch = sess.run(next_batch)

                    # And run the training op
                    acc,cost=sess.run([accuracy,loss], feed_dict={model.x: img_batch,
                                                  model.y_gt: label_batch,
                                                  keep_prob: 0.7})
                    # Generate summary with the current batch of data and write to file
                    if step % display_step == 0:
                        s = sess.run(merged_summary, feed_dict={model.x: img_batch,
                                                                model.y_gt: label_batch,
                                                                keep_prob: 0.7})

                        writer.add_summary(s, epoch*train_batches_per_epoch + step)
                    train_acc += acc
                    train_loss += cost
                    train_count += 1
                pbar.update(1)
                pbar.set_description("Training : "+str(epoch)+
                                     " Accuracy : "+str(train_acc/train_count if train_count !=0 else 0)+
                                     " Loss : "+str(train_loss/train_count if train_count !=0 else 0))
                # Validate the model on the entire validation set

                sess.run(validation_init_op)
                test_acc = 0.
                test_count = 0
                for step in range(val_batches_per_epoch):

                    img_batch, label_batch = sess.run(next_batch)
                    acc = sess.run(accuracy, feed_dict={model.x: img_batch,
                                                        model.y_yt: label_batch,
                                                        keep_prob: 1.})
                    test_acc += acc
                    test_count += 1
                if epoch % 20 == 0 and epoch!=0 and test_count!=0:
                    print("Validation : "+str(epoch+1)+" Accuracy : "+str(test_acc/test_count))

                # save checkpoint of the model
                checkpoint_name = os.path.join(checkpoint_path, 'model_epoch'+str(epoch+1)+'.ckpt')
                save_path = saver.save(sess, checkpoint_name)

Training : 67 Accuracy : 0.0 Loss : 2.633448362350464:   7%|▋         | 68/1000 [07:37<1:41:34,  6.54s/it] 